In [1]:
import os
import numpy as np
from numpy import unicode
import h5py
import glob
import cv2
from PIL import Image
import tensorflow as tf
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import keras.backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Lambda, Input, Dense, GlobalAveragePooling2D, Merge, Dropout
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from imutils import build_montages
import multiprocessing as mp
import math
import tqdm
#from annoy import AnnoyIndex
import json
import PIL
from functools import partial, update_wrapper

Using TensorFlow backend.


In [ ]:
img_size = 224     # input image size for network
margin = 0.3       # margin for triplet loss
batch_size = 16    # size of mini-batch
num_triplet = 1000 
valid_frequency = 100
num_epoch = 200 

In [2]:
def get_triplets(data, labels):
    pos_label, neg_label = np.random.choice(labels, 2, replace=False)
    pos_indexes = np.where(labels == pos_label)[0]
    neg_indexes = np.where(labels == neg_label)[0]
    np.random.shuffle(pos_indexes)
    np.random.shuffle(neg_indexes)
    anchor = data[pos_indexes[0]]
    positive = data[pos_indexes[-1]]
    negative = data[neg_indexes[0]]
    return (anchor, positive, negative)

In [ ]:
# utility function to freeze some portion of a function's arguments
def wrapped_partial(func, *args, **kwargs):
    partial_func = partial(func, *args, **kwargs)
    update_wrapper(partial_func, func)
    return partial_func

# calculate recall score
def recall(y_true, y_pred):
    return min(len(set(y_true) & set(y_pred)), 1)

# margin triplet loss
def margin_triplet_loss(y_true, y_pred, margin, batch_size):
    out_a = tf.gather(y_pred, tf.range(0, batch_size, 3))
    out_p = tf.gather(y_pred, tf.range(1, batch_size, 3))
    out_n = tf.gather(y_pred, tf.range(2, batch_size, 3))
    
    loss = K.maximum(margin
                 + K.sum(K.square(out_a-out_p), axis=1)
                 - K.sum(K.square(out_a-out_n), axis=1),
                 0.0)
    return K.mean(loss)

In [ ]:
class SemiHardNegativeSampler:
    def __init__(self, pool, batch_size, num_samples):
        self.pool = pool
        self.batch_size = batch_size
        self.num_samples = num_samples
        self.resample()
        
    # sample triplets with semi-hard negatives
    def resample(self):        
        sample_classes = np.random.choice(np.arange(X_train_user.shape[0]), p=num_images_by_class/num_images_by_class.sum(), size=self.num_samples)

        sample_images = []
        for i in sample_classes:
            sample_images.append(train_images_user[np.random.choice(X_train_user[i].nonzero()[1], replace=False)])
            sample_images.append(train_images_seller[np.random.choice(X_train_seller[i].nonzero()[1], replace=False)])
        sample_images = np.array(sample_images)

        pred_sample = model.predict_generator(batch_generator_predict(pool, 32, sample_images), math.ceil(len(sample_images)/32), max_q_size=1, workers=1)

        a = pred_sample[np.arange(0, len(pred_sample), 2)]
        p = pred_sample[np.arange(1, len(pred_sample), 2)]
        triplets = []
        self.dists = []

        for i in range(self.num_samples):
            d = np.square(a[i] - p[i]).sum()
            neg_sample_classes = (sample_classes != sample_classes[i]).nonzero()[0]

            neg = p[neg_sample_classes]

            neg_ids = sample_images.reshape((-1, 2))[neg_sample_classes, 1]

            d_neg = np.square(neg - a[i]).sum(axis=1)

            semihard = np.where(d_neg > d)[0]

            if len(semihard) == 0:
                n = np.argmax(d_neg)
            else:
                n = semihard[np.argmin(d_neg[semihard])]
                
            self.dists.append(d_neg[n]-d)

            triplets.append(np.concatenate([sample_images.reshape((-1, 2))[i], np.array([neg_ids[n]])]))

        self.triplets = np.array(triplets)
        
    # data generator for triplets
    def batch_generator(self):
        i = 0
        while True:
            batch = self.triplets[i:i+self.batch_size//3].ravel()
            
            i += self.batch_size//3
            if len(batch) == 0:
                yield np.zeros((0, img_size, img_size, 3))
            else:
                result = pool.map(preprocess_image_worker_aug, batch)
                X_batch = np.concatenate(result, axis=0)
                yield X_batch, np.zeros(len(batch))

    # return data generator for triplets
    def get_generator(self):
        gen = self.batch_generator()
        return gen

In [3]:
def dump_features(image_paths, labels, hdf5_path, feature_extractor, image_formats=("jpg")):
    # image_paths = []
    # for image_format in image_formats:
    #     image_paths += glob.glob("{}/*.{}".format(images_dir, image_format))
    # image_paths = sorted(image_paths)
    db = h5py.File(hdf5_path, mode="w")
    features_shape = ((len(labels),), feature_extractor.output_shape[1:])
    features_shape = [dim for sublist in features_shape for dim in sublist]
    imageIDDB = db.create_dataset("image_ids", shape=(len(labels),),
                                  dtype=h5py.special_dtype(vlen=unicode))
    featuresDB = db.create_dataset("features",
                                   shape=features_shape, dtype="float")
    labelsDB = db.create_dataset("labels",
                                 shape=(len(labels),), dtype="int")
    for i in range(0, len(labels), 16):
        start,end = i, i+16
        image_ids = [path.split("/")[-1] for path in image_paths[start:end]]
        images = [cv2.imread(path,1) for path in image_paths[start:end]]
        features = feature_extractor.extract(images)
        imageIDDB[start:end] = image_ids
        featuresDB[start:end] = features
        labelsDB[start:end] = labels[start:end]
        print("Extracting {}/{}".format(i+1+16, len(labels)))
    db.close()

In [4]:
def extract_features(hdf5_path):
    db = h5py.File(hdf5_path,mode="r")
    features = db["features"][:]
    labels = db["labels"][:]

    return (features, labels)

def extract_embeddings(features, model):
    embeddings = model.predict([features, features, features])
    return embeddings[:,:,0]

In [5]:
def euclidean_distance(a,b):
    return K.sqrt(K.sum(K.square((a-b)), axis=1))

def cosine_distance(a, b, normalize=True):
    if normalize:
        a = K.l2_normalize(a, axis=0)
        b = K.l2_normalize(b, axis=0)
    return K.prod(K.stack([a, b], axis=1), axis=1)

def triplet_loss(y_true, anchor_positive_negative_tensor):
    anchor = anchor_positive_negative_tensor[:,:,0]
    positive = anchor_positive_negative_tensor[:,:,1]
    negative = anchor_positive_negative_tensor[:,:,2]
    Dp = euclidean_distance(anchor, positive)
    Dn = euclidean_distance(anchor, negative)

    return K.maximum(0.0, 1+K.mean(Dp-Dn))

In [6]:
class ImageNetFeatureExtractor(object):
    def __init__(self, model="vgg16", resize_to=(150, 150)):
        # MODEL_DICT = {"vgg16": VGG16, "vgg19": VGG19, "inception": InceptionV3, "resnet": ResNet50,
        #               "xception": Xception}
        # network = MODEL_DICT[model.lower()]
        self.model_name = model.lower()
        self.model = self.getModel()
        self.preprocess_input = preprocess_input
        self.imageSize = resize_to

    def extract(self, images):
        images = self.preprocess(images)
        return self.model.predict(images)

    def getModel(self):
        model = load_model('C:/Users/hp/Downloads/data/model/top_model_vgg/final_model.h5')
        intermediate_layer_model = Model(inputs=model.input,
                                         outputs=model.get_layer("block5_pool").output)

        return intermediate_layer_model

    @property
    def output_shape(self):
        return self.model.compute_output_shape([[None, self.imageSize[0], self.imageSize[1], 3]])

    def resize_images(self, images):
        images = np.array([cv2.resize(image, (self.imageSize[0], self.imageSize[1])) for image in images])
        return images

    def preprocess(self, images):
        images = self.resize_images(images)
        images = self.preprocess_input(images.astype("float"))
        return images

def concat_tensors(tensors, axis=-1):
    return K.concatenate([K.expand_dims(t, axis=axis) for t in tensors])


def get_small_network(input_shape=(None, 4, 4, 512)):
    model = Sequential()
    model.add(GlobalAveragePooling2D(input_shape=input_shape[1:]))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation="relu"))
    #model.add(Dense(128, activation="relu"))
    return model

def get_triplet_network(input_shape=(None, 4, 4, 512)):
    base_model = get_small_network(input_shape=input_shape)

    anchor_input = Input(input_shape[1:])
    positive_input = Input(input_shape[1:])
    negative_input = Input(input_shape[1:])

    anchor_embeddings = base_model(anchor_input)
    positive_embeddings = base_model(positive_input)
    negative_embeddings = base_model(negative_input)

    output = Lambda(concat_tensors)([anchor_embeddings, positive_embeddings, negative_embeddings])
    model = Model([anchor_input, positive_input, negative_input], output)

    return model

In [7]:
image_path_list=[]
labels=[]
train_data_dir = 'C:/Users/hp/Downloads/data/train/'
cat_root = train_data_dir + 'cats' + '/'
cats=os.listdir(cat_root)

for cat in cats:
    image_path_list.append(cat_root + cat)
    labels.append(0)

dog_root = train_data_dir + 'dogs' + '/'
dogs=os.listdir(dog_root)
for dog in dogs:
    image_path_list.append(dog_root + dog)
    labels.append(1)

labels = np.array(labels)

In [11]:
images = [cv2.imread(path,1) for path in image_path_list[0:10]]

In [12]:
images

[array([[[ 87, 164, 203],
         [ 87, 164, 203],
         [ 88, 165, 204],
         ...,
         [122, 201, 240],
         [121, 200, 239],
         [120, 199, 238]],
 
        [[ 87, 164, 203],
         [ 87, 164, 203],
         [ 88, 165, 204],
         ...,
         [123, 202, 241],
         [122, 201, 240],
         [120, 199, 238]],
 
        [[ 87, 164, 203],
         [ 87, 164, 203],
         [ 88, 165, 204],
         ...,
         [123, 202, 241],
         [122, 201, 240],
         [121, 200, 239]],
 
        ...,
 
        [[ 55, 122, 153],
         [ 55, 122, 153],
         [ 55, 122, 153],
         ...,
         [  0,   2,   2],
         [  0,   2,   2],
         [  0,   2,   2]],
 
        [[ 54, 121, 152],
         [ 54, 121, 152],
         [ 54, 121, 152],
         ...,
         [  0,   2,   2],
         [  0,   2,   2],
         [  0,   2,   2]],
 
        [[ 53, 120, 151],
         [ 53, 120, 151],
         [ 53, 120, 151],
         ...,
         [  0,   1,   1],
  

In [8]:
feature_extractor = ImageNetFeatureExtractor(model='vgg16')

In [9]:
print ("[+] Successfully loaded pre-trained model")
dump_features(image_path_list, labels=np.array(labels),
              hdf5_path='C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5', feature_extractor=feature_extractor,
              image_formats=("jpg","png"))

[+] Successfully loaded pre-trained model
Extracting 17/1200
Extracting 33/1200
Extracting 49/1200
Extracting 65/1200
Extracting 81/1200
Extracting 97/1200
Extracting 113/1200
Extracting 129/1200
Extracting 145/1200
Extracting 161/1200
Extracting 177/1200
Extracting 193/1200
Extracting 209/1200
Extracting 225/1200
Extracting 241/1200
Extracting 257/1200
Extracting 273/1200
Extracting 289/1200
Extracting 305/1200
Extracting 321/1200
Extracting 337/1200
Extracting 353/1200
Extracting 369/1200
Extracting 385/1200
Extracting 401/1200
Extracting 417/1200
Extracting 433/1200
Extracting 449/1200
Extracting 465/1200
Extracting 481/1200
Extracting 497/1200
Extracting 513/1200
Extracting 529/1200
Extracting 545/1200
Extracting 561/1200
Extracting 577/1200
Extracting 593/1200
Extracting 609/1200
Extracting 625/1200
Extracting 641/1200
Extracting 657/1200
Extracting 673/1200
Extracting 689/1200
Extracting 705/1200
Extracting 721/1200
Extracting 737/1200
Extracting 753/1200
Extracting 769/1200
Extr

In [20]:
model_check_point_loc = 'C:/Users/hp/Downloads/data/similarity/vgg16_cats_dogs.h5'

In [21]:
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')
print("[+] Finished loading extracted features")
model = get_triplet_network(features.shape)
data = []
for i in range(len(features)):
    anchor, positive, negative = get_triplets(features, labels)
    data.append([anchor, positive, negative])
data = np.array(data)
#  1200 = training examples
# 256 = # of features
# tripple of images - anchor, positive and negatives = 3
targets = np.zeros(shape=(1200, 256, 3))
callback = ModelCheckpoint(model_check_point_loc, period=1, monitor="val_loss")
X_train, X_test, Y_train, Y_test = train_test_split(data, targets)
model.compile(optimizers.Adam(1e-4), triplet_loss)
model.fit([X_train[:,0], X_train[:,1], X_train[:,2]], Y_train, epochs=10,
          validation_data=([X_test[:,0], X_test[:,1], X_test[:,2]], Y_test),
          callbacks=[callback], batch_size=16)

[+] Finished loading extracted features
Train on 900 samples, validate on 300 samples
Epoch 1/10
900/900 [==============================] - 2s 2ms/step - loss: 1.6853 - val_loss: 0.1812
Epoch 2/10
900/900 [==============================] - 1s 1ms/step - loss: 1.5027 - val_loss: 0.0608
Epoch 3/10
900/900 [==============================] - 1s 1ms/step - loss: 0.4913 - val_loss: 0.0225
Epoch 4/10
900/900 [==============================] - 1s 1ms/step - loss: 0.7786 - val_loss: 0.0121
Epoch 5/10
900/900 [==============================] - 1s 1ms/step - loss: 0.1222 - val_loss: 0.0000e+00
Epoch 6/10
900/900 [==============================] - 1s 1ms/step - loss: 0.6268 - val_loss: 0.0000e+00
Epoch 7/10
900/900 [==============================] - 1s 1ms/step - loss: 0.2652 - val_loss: 0.0038
Epoch 8/10
900/900 [==============================] - 1s 1ms/step - loss: 0.2103 - val_loss: 0.0000e+00
Epoch 9/10
900/900 [==============================] - 1s 1ms/step - loss: 0.1575 - val_loss: 0.0000e+0

In [22]:
image_ids = h5py.File('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5', mode="r")["image_ids"][:]

In [23]:
def get_image_index(imagePath='C:/Users/hp/Downloads/data/train/cats/cat.1.jpg'):
    filename = imagePath.split("/")[-1]
    return np.where(image_ids == filename)[0][0]

def get_image_path(index):
    for imagePath in image_path_list:
        if imagePath.rsplit('/', 1)[1] == image_ids[index]:
            return imagePath
    #return args["dataset"].strip("/")+"/"+str(image_ids[index])

In [24]:
def get_image_path(index):
    for imagePath in image_path_list:
        if imagePath.rsplit('/', 1)[1] == image_ids[index]:
            return imagePath

In [25]:
model = load_model(model_check_point_loc, custom_objects={"triplet_loss":triplet_loss})
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')
embeddings = model.predict([features, features, features])
embeddings = embeddings[:,:,2]

In [26]:
image_id = get_image_index()
query = embeddings[image_id]
distances = pairwise_distances(query.reshape(1,-1), embeddings)
indices = np.argsort(distances)[0][:12]
images = [cv2.imread(get_image_path(index)) for index in indices]
images = [cv2.resize(image, (200,200)) for image in images]
result = build_montages(images, (200, 200), (4,3))[0]
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
embeddings

array([[ 2.5538974,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       , 12.41322  ,  0.8122095, ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       , 13.904363 ,  3.201221 , ...,  3.0065172,  0.       ,
         0.       ],
       ...,
       [ 9.10835  ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [10.256804 ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [16.977217 ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [28]:
source = 'C:/Users/hp/Downloads/data/validation/cats/cat.1106.jpg'
img = image.load_img(source, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
# get the features 
extract_features = feature_extractor.getModel().predict(x)

In [29]:
test_embedding = model.predict([extract_features, extract_features, extract_features])

In [30]:
test_query = test_embedding[:,:,2][0]
distances = pairwise_distances(test_query.reshape(1,-1), embeddings)
indices = np.argsort(distances)[0][:12]
images = [cv2.imread(get_image_path(index)) for index in indices]
images = [cv2.resize(image, (200,200)) for image in images]
result = build_montages(images, (200, 200), (4,3))[0]
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
def get_similar_image_indices(embeddings, index, num_results=4):
    query = embeddings[index]
    distances = pairwise_distances(query.reshape(1, -1), embeddings)
    indices = np.argsort(distances)[0][:num_results]
    return indices

def find_num_correct(true_indices, predicted_indices):
    num_correct = 0
    for i in true_indices:
        if i in predicted_indices:
            num_correct += 1
    return num_correct

model = load_model(model_check_point_loc, custom_objects={"triplet_loss":triplet_loss})
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')
embeddings = model.predict([features, features, features])
embeddings = embeddings[:,:,2]
num_correct = 0

for i in range(1200):
    similar_indices = get_similar_image_indices(embeddings, i)
    true_indices = np.where(labels==(i/4)+1)[0].tolist()
    num_correct += find_num_correct(true_indices, similar_indices)/4.0
print "Accuracy", num_correct/1000.0